In [1]:
from math import sin, cos, sqrt, atan2, atan, radians
from itertools import compress
from datetime import timedelta
from datetime import datetime
from astropy.io import ascii
from os import listdir
import urllib.request
import pandas as pd
import numpy as np
import requests
import shutil
import time
import math
import gzip
import json
import os 

from pyspark.sql.session import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

In [2]:
##############################################################################################################
def find_files( path_to_dir, suffix=".txt" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

##############################################################################################################
#Snow/SWE observing thresholds
Snowfall_TH    = 50.8 #mm
SnowWaterEq_TH = 2.8 #mm

#Anselin Morans I: Detect spatial outliers and remove them. Coords must bein lat/lon
#Constants
RadEarth   = 6371000 #m Euclidean Dist.

def Outlier_Check(XYZ,dz,pwr):
    dsize    = XYZ.shape
    n        = int((dsize[0]))
    print("All available observations pre-outlier analysis: " + str(n))
    #Pre-alo Variables
    IDW      = np.zeros((n,n))
    widw     = np.zeros((n,n))
    hij      = np.zeros((n,n))
    #Loop through each point
    for i in range(n):
        #Determine euclidean distance (km) from each observation [RadEarth*c/1000]
        LonD        = np.radians(XYZ[:,0])
        LatD        = np.radians(XYZ[:,1])
        delta_phi   = LatD[i]-LatD
        delta_lamda = LonD[i]-LonD
        a           = np.sin(delta_phi/2)**2 + np.cos(LatD[i])*np.cos(LatD)*np.sin(delta_lamda/2.0)**2
        a[a==0]     = np.nan
        c           = 2*np.arctan(np.sqrt(a)/np.sqrt(1-a)) #getting errors in a
        #Replace these values with nan... avoid error message
        c[c==0]     = np.nan
        #Determine Inverse Distance Weight between each observation
        IDW[i,:]    = 1/(RadEarth*c/1000)**pwr
        IDW[i,i]    = np.nan
        #Index observations for each stations
        widw[:,i]   = XYZ[i,2]
        widw[i,i]   = np.nan
        #Determine Euclidean distance b/n each station
        hij[i,:]    = RadEarth*c/1000
        hij[i,i]    = 99999
    #Are the observations w/n threshold that has been defined
    Eucl_Logical    = hij<dz #getting errors in hij
    Sum_row_Eucl    = np.nansum(IDW*Eucl_Logical, axis=1)
    #Weighted distance b/n observations as a ratio
    Wij = np.full((n, n),np.nan)
    for i in range(n):
        if Sum_row_Eucl[i] > 0:
            Wij[i,:] = np.round(IDW[i,:]/Sum_row_Eucl[i],2)        
        Wij[i,i] = np.nan
    #Observations with no cooresponding stations within a given threshold will have Wij == inf... 
    #Replace these values with nan
    std_idw_p             = np.std(XYZ[:,2])
    #widw_std = widw*Eucl_Logical
    #widw_std[widw_std==0] = np.nan
    #std_idw  = np.nanstd(widw_std,axis=1)
    #std_idw[std_idw==0] = std_idw_p
    std_idw           = np.std(XYZ[:,2])
    #std_idw           = np.std(Wij*widw*Eucl_Logical,axis=1)
    weighted_variable = np.nansum(Wij*widw*Eucl_Logical,axis=1)
    #Z-score for a normal distribution
    Zscore_i          = (XYZ[:,2]-weighted_variable)/std_idw
    #non_outliers      = abs(Zscore_i)<1.645
    #Z-score for a log distribution
    #Zscore_i          = np.exp((XYZ[:,2]-weighted_variable)/std_idw)
    Loggical_arraay   = [np.logical_and(Zscore_i > -1.280, Zscore_i < 100)] #v == 4

    #Loggical_arraay   = [np.logical_and(Zscore_i > 0.711, Zscore_i < 9.488)] #v == 4
    #Loggical_arraay   = [np.logical_and(Zscore_i > 2.733, Zscore_i < 15.507)] #v == 12
    #Return outliers
    XYZ_outliers      = XYZ[Loggical_arraay[0]==False]
    #Return nonoutliers
    XYZ_nonoutliers   = XYZ[Loggical_arraay[0]==True]
    print("Outliers detected: " + str(len(XYZ_outliers)))
    print("Specifications: Interpolation == IDW to the " + str(pwr) + " power")
    print("Specifications: Search Radius == " + str(dz) + "km")
    print("----------------------------------------------")
    return(XYZ_nonoutliers,XYZ_outliers,Zscore_i)

def Density_Check(lons,lats,data,dz,dp):
    XYZ      = np.concatenate((np.expand_dims(lons,1), np.expand_dims(lats,1), np.expand_dims(data,1)), axis=1)
    print("----------------------------------------------")
    print("Initial number of observations: " + str(len(XYZ)))
    n        = len(XYZ)
    #Pre-alo Variables
    hij      = np.zeros((n,n))
    #Loop through each point
    for i in range(n):
        #Determine euclidean distance (km) from each observation [RadEarth*c/1000]
        LonD        = np.radians(XYZ[:,0])
        LatD        = np.radians(XYZ[:,1])
        delta_phi   = LatD[i]-LatD
        delta_lamda = LonD[i]-LonD
        a           = np.sin(delta_phi/2)**2 + np.cos(LatD[i])*np.cos(LatD)*np.sin(delta_lamda/2.0)**2
        a[a==0]     = np.nan
        c           = 2*np.arctan(np.sqrt(a)/np.sqrt(1-a)) #getting errors in a
        #Replace these values with nan... avoid error message
        c[c==0]     = np.nan
        #Determine Euclidean distance b/n each station
        hij[i,:]    = RadEarth*c/1000
        hij[i,i]    = 99999
    #Are the observations w/n threshold that has been defined
    Eucl_Logical    = hij<dz #getting errors in hij
    Eucl_Logical    = np.expand_dims(np.sum(Eucl_Logical,axis=1),1)
    XYZ             = np.asarray(list(compress(XYZ,(Eucl_Logical>dp)==True)))
    print("Clustered number of observations: " + str(int(len(XYZ))))
    print("Specifications: Distance == " + str(dz) + "km Point Density == " + str(dp))
    print("----------------------------------------------")
    return(XYZ)

def Dumby_Check(lons, lats, query_lon, query_lat, dz):
    hij        = np.zeros((len(lons),1))
    for z in range(len(lons)):
        # Constant
        R = RadEarth
        # Observation points
        lat1 = lats[z]
        lon1 = lons[z]
        # Query/grid point
        lat2 = query_lat
        lon2 = query_lon
        # Calculation
        phi1, phi2 = math.radians(lat1), math.radians(lat2) 
        dphi       = math.radians(lat2 - lat1)
        dlambda    = math.radians(lon2 - lon1)
        a          = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
        # Euclidean distance in km
        hij[z,0]   = ((2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a)))/1000)
    hij            = np.sum(hij<=dz)
    # Return the number of stations within dz kms of an observation.
    return(hij)

def Dumby_Check_02(XYZ, query_lon, query_lat, dz):
    hij        = np.zeros((len(XYZ),1))
    for z in range(len(XYZ)):
        # Constant
        R = RadEarth
        # Observation points
        lat1 = XYZ[z,1]
        lon1 = XYZ[z,0]
        # Query/grid point
        lat2 = query_lat
        lon2 = query_lon
        # Calculation
        phi1, phi2 = math.radians(lat1), math.radians(lat2) 
        dphi       = math.radians(lat2 - lat1)
        dlambda    = math.radians(lon2 - lon1)
        a          = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
        # Euclidean distance in km
        hij[z,0]   = ((2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a)))/1000)
    XYZ = XYZ[hij[:,0]<dz]
    # Return the number of stations within dz kms of an observation.
    return(XYZ)

def simple_idw(x, y, z, xi, yi):
    dist = distance_matrix(x,y, xi,yi)

    # In IDW, weights are 1 / distance
    weights = 1.0 / dist

    # Make weights sum to one
    weights /= weights.sum(axis=0)

    # Multiply the weights for each interpolated point by all observed Z-values
    zi = np.dot(weights.T, z)
    return zi

def distance_matrix(x0, y0, x1, y1):
    obs = np.vstack((x0, y0)).T
    interp = np.vstack((x1, y1)).T

    # Make a distance matrix between pairwise observations
    # Note: from <http://stackoverflow.com/questions/1871536>
    # (Yay for ufuncs!)
    d0 = np.subtract.outer(obs[:,0], interp[:,0])
    d1 = np.subtract.outer(obs[:,1], interp[:,1])

    return np.hypot(d0, d1)

def YYYYMMDD_string(Datetime_Value):
    if Datetime_Value.month>9: Month_str = str(Datetime_Value.month)
    else: Month_str = "0"+str(Datetime_Value.month)   
    if Datetime_Value.day>9: Day_str = str(Datetime_Value.day)
    else: Day_str = "0"+str(Datetime_Value.day)
    return (str(Datetime_Value.year)+Month_str+Day_str)

In [ ]:
for i in range(len(event_start)):
    if event_start[i][2] != event_end[i][2]:
        print(event_start[i])
        print(event_end[i])
        
event_start = [[12,30,2013],[12,28,2009],[12,28,2000],[12,31,1970],[12,31,1924]]
event_end   = [[1,3,2014]  ,[1,4,2010]  ,[1,1,2001]  ,[1,2,1971]  ,[1,3,1925]  ]

In [ ]:
event_start = [[10,29,2020],[11,29,2019],[1,18,2019],[11,14,2018],[3,20,2018],[3,11,2018],[3,5,2018],[3,1,2018],[1,3,2018],
                [3,12,2017],[2,9,2017],[11,17,2016],[1,22,2016],[2,14,2015],[2,8,2015],[1,29,2015],[1,25,2015],
                [12,9,2014],[11,26,2014],[2,11,2014],[1,20,2014],[12,30,2013],[12,13,2013],[3,17,2013],[3,3,2013],
                [2,8,2013],[12,28,2012],[12,24,2012],[10,25,2011],[2,24,2011],[2,1,2011],[1,26,2011],[1,9,2011],
                [12,24,2010],[2,21,2010],[2,12,2010],[2,8,2010],[2,4,2010],[12,28,2009],[12,18,2009],[12,7,2009],
                [2,26,2009],[2,22,2009],[12,21,2008],[12,18,2008],[12,14,2007],[11,30,2007],[4,3,2007],[3,16,2007],
                [2,11,2007],[2,10,2006],[2,28,2005],[1,22,2005],[12,4,2003],[2,14,2003],[1,1,2003],[12,23,2002],[12,28,2000],[2,16,2000],[1,24,2000],[1,24,2000],
                [3,12,1999],[1,13,1999],[3,31,1997],[1,8,1997],[4,9,1996],[3,3,1996],[2,1,1996],[1,6,1996],[12,18,1995],
                [2,2,1995],[2,28,1994],[2,22,1994],[1,16,1994],[1,4,1994],[1,1,1994],[3,12,1993],[2,20,1993],[2,14,1993],
                [12,9,1992],[12,27,1990],[2,9,1988],[1,22,1988],[1,5,1988],[12,13,1987],[11,10,1987],[2,22,1987],[1,21,1987],
                [1,8,1987],[1,1,1987],[3,1,1985],[1,29,1985],[2,24,1984],[2,10,1983],[4,4,1982],[1,20,1982],[1,12,1982],
                [2,17,1979],[2,6,1979],[2,4,1978],[1,17,1978],[1,14,1978],[1,11,1978],[3,21,1977],[1,7,1977],[1,8,1974],
                [12,15,1973],[2,16,1972],[11,23,1971],[2,26,1971],[12,31,1970],[12,25,1969],[2,25,1969],[2,22,1969],[2,8,1969],
                [3,20,1967],[2,6,1967],[12,22,1966],[2,23,1966],[1,28,1966],[1,21,1966],[2,18,1964],[1,9,1964],[12,21,1963],
                [3,5,1962],[2,28,1962],[2,13,1962],[2,1,1961],[1,18,1961],[12,10,1960],[2,29,1960],[2,12,1960],[3,12,1959],
                [3,18,1958],[2,12,1958],[12,3,1957],[3,18,1956],[3,14,1956],[3,3,1956],[2,17,1952],[12,13,1951],[11,22,1950],
                [2,11,1950],[1,29,1949],[1,23,1948],[12,25,1947],[2,27,1947],[2,20,1947],[2,15,1946],[12,17,1945],[1,13,1945],
                [12,8,1944],[2,8,1944],[1,25,1943],[3,28,1942],[2,28,1942],[3,7,1941],[2,13,1940],[11,23,1938],[2,10,1936],
                [1,18,1936],[1,21,1935],[2,23,1934],[12,26,1933],[12,16,1932],[3,4,1931],[12,19,1929],[2,20,1929],[2,16,1927],
                [2,3,1926],[1,7,1926],[1,28,1925],[12,31,1924],[4,1,1924],[2,17,1924],[2,3,1923],[1,26,1922],[2,18,1921],[2,4,1920],
                [1,25,1918],[1,21,1918],[1,12,1918],[12,12,1917],[12,6,1917],[3,2,1917],[3,2,1916],[12,10,1915],[4,3,1915],[3,2,1915],
                [1,29,1915],[2,12,1914],[2,16,1910],[2,10,1910],[1,12,1910],[12,23,1909],[3,2,1909],[1,27,1909],[1,10,1909],[2,16,1908],
                [2,3,1908],[1,29,1908],[2,4,1907],[3,17,1906],[3,12,1906],[1,27,1904],[2,14,1903],[12,11,1902],[3,3,1902],[2,13,1902],
                [2,1,1901],[3,15,1900],[2,26,1900]]
event_end   = [[10,30,2020],[12,3,2019],[1,21,2019],[11,16,2018],[3,22,2018],[3,15,2018],[3,8,2018],[3,3,2018],[1,5,2018],
                [3,15,2017],[2,10,2017],[11,22,2016],[1,24,2016],[2,16,2015],[2,10,2015],[2,3,2015],[1,28,2015],
                [12,14,2014],[11,28,2014],[2,14,2014],[1,22,2014],[1,3,2014],[12,16,2013],[3,20,2013],[3,9,2013],
                [2,10,2013],[12,31,2012],[12,28,2012],[10,31,2011],[2,27,2011],[2,4,2011],[1,27,2011],[1,13,2011],
                [12,28,2010],[3,1,2010],[2,19,2010],[2,11,2010],[2,8,2010],[1,4,2010],[12,21,2009],[12,11,2009],
                [3,3,2009],[2,24,2009],[12,23,2008],[12,22,2008],[12,17,2007],[12,4,2007],[4,6,2007],[3,18,2007],
                [2,16,2007],[2,14,2006],[3,2,2005],[1,24,2005], [12,8,2003],[2,18,2003],[1,4,2003],[12,26,2002],[1,1,2001],[2,20,2000],[2,1,2000],[1,27,2000],
                [3,16,1999],[1,16,1999],[4,1,1997],[1,12,1997],[4,11,1996],[3,9,1996],[2,4,1996],[1,9,1996],
                [12,22,1995],[2,6,1995],[3,4,1994],[2,25,1994],[1,18,1994],[1,9,1994],[1,5,1994],[3,15,1993],
                [2,24,1993],[2,18,1993],[12,13,1992],[12,29,1990],[2,14,1988],[1,27,1988],[1,9,1988],[12,17,1987],
                [11,12,1987],[2,24,1987],[1,24,1987],[1,12,1987],[1,3,1987],[3,5,1985],[2,3,1985],[3,1,1984],
                [2,13,1983],[4,8,1982],[1,24,1982],[1,15,1982],[2,20,1979],[2,9,1979],[2,8,1978],[1,21,1978],
                [1,19,1978],[1,15,1978],[3,25,1977],[1,11,1977],[1,12,1974],[12,18,1973],[2,20,1972],[11,27,1971],
                [3,6,1971],[1,2,1971],[12,29,1969],[3,4,1969],[2,28,1969],[2,10,1969],[3,23,1967],[2,8,1967],
                [12,26,1966],[2,26,1966],[2,1,1966],[1,24,1966],[2,21,1964],[1,14,1964],[12,24,1963],[3,9,1962],
                [3,7,1962],[2,16,1962],[2,5,1961],[1,21,1961],[12,13,1960],[3,5,1960],[2,15,1960],[3,14,1959],
                [3,23,1958],[2,18,1958],[12,5,1957],[3,20,1956],[3,17,1956],[3,9,1956],[2,18,1952],[12,16,1951],
                [11,30,1950],[2,17,1950],[2,1,1949],[1,25,1948],[12,27,1947],[3,4,1947],[2,24,1947],[2,21,1946],
                [12,20,1945],[1,17,1945],[12,13,1944],[2,13,1944],[1,29,1943],[3,31,1942],[3,4,1942],[3,10,1941],
                [2,15,1940],[11,25,1938],[2,15,1936],[1,21,1936],[1,25,1935],[2,27,1934],[12,27,1933],[12,18,1932],
                [3,12,1931],[12,24,1929],[2,22,1929],[2,21,1927],[2,5,1926],[1,10,1926],[1,31,1925],[1,3,1925],
                [4,4,1924],[2,21,1924],[2,7,1923],[1,30,1922],[2,22,1921],[2,7,1920],[1,29,1918],[1,23,1918],
                [1,16,1918],[12,15,1917],[12,9,1917],[3,6,1917],[3,9,1916],[12,15,1915],[4,5,1915],[3,8,1915],
                [2,3,1915],[2,15,1914],[2,18,1910],[2,13,1910],[1,15,1910],[12,27,1909],[3,5,1909],[1,31,1909],
                [1,15,1909],[2,20,1908],[2,7,1908],[2,2,1908],[2,6,1907],[3,20,1906],[3,17,1906],[1,30,1904],
                [2,18,1903],[12,14,1902],[3,6,1902],[2,19,1902],[2,6,1901],[3,16,1900],[3,3,1900]]
print(len(event_start),len(event_end))
#print(event_start[20],event_end[20])
#event_start = list(event_start[20])
#event_end   = list(event_end[20])
#add the access token you got from NOAA
Variable      = 'SNOW' #SNOW,PRCP,TAVG 
Parent_dir    = 'C:/Users/Mike/NESIS_Kriging/'
# Overwrite file if date isn't available in previous csv file
Overwrite  =  0 #1==yes 0==no
# Analysis Domain
North      = 55.0
East       = -60.0
South      = 25.0
West       = -110.0
# Dumby Grid
grid_space     = 0.25
grid_lon       = np.arange(West , East , grid_space)
grid_lat       = np.arange(South, North, grid_space) #grid_space is the desired delta/step of the output array 
xintrp, yintrp = np.meshgrid(grid_lon, grid_lat)
#############################################################################################################
# Retrieve station metadata if not obtained
GHCND_Base_FTP   = 'ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/'
Station_File     = 'ghcnd-stations.txt'
Archive_Dir      = 'C:/Users/Mike/Desktop/GHCND/AirMG/'
List_Archive_Dir = find_files( Archive_Dir, suffix=".txt" )
print("------------------------------------------------------------------------")
if Station_File not in List_Archive_Dir:
    print("Downloading "+Station_File)
    Station_File_URL = GHCND_Base_FTP + Station_File
    urllib.request.urlretrieve(Station_File_URL, Station_File)
    shutil.move(Station_File,Archive_Dir)
else: print(Station_File+" is Downloaded: "+Archive_Dir)
#############################################################################################################
# Parse the station metadata
Station_File         = 'ghcnd-stations.txt'
with open(Archive_Dir+Station_File) as f:
    Station_Metadata = f.readlines()
Station_Metadata     = [x.strip() for x in Station_Metadata] 
Station_Metadata_np  = np.empty([len(Station_Metadata),4], dtype=object)
for x in range(len(Station_Metadata)):
    query                    = Station_Metadata[x]
    Station_Metadata_np[x,0] = query[0:11].strip()  #Station I D
    Station_Metadata_np[x,1] = query[12:20].strip() #Lat
    Station_Metadata_np[x,2] = query[21:30].strip() #Lon
    # Determine which stations fit within the analysis domain
    if (float(query[12:20].strip()) < North) and (float(query[12:20].strip()) > South) and \
       (float(query[21:30].strip()) < East) and (float(query[21:30].strip()) > West): Station_Metadata_np[x,3] = '1'
    else: Station_Metadata_np[x,3] = '0'
print("------------------------------------------------------------------------")
print(Station_File+" file has been parsed!")
#Column headers for spark dataframe
schema = StructType([
        StructField("Station", StringType()),
        StructField("YYYYMMDD", StringType()),
        StructField("Type", StringType()),
        StructField("VALUE", DoubleType()),
        StructField("MFLAG1", StringType()),
        StructField("QFLAG1", StringType()),
        StructField("SFLAG1", StringType()),
        StructField("ObsTime", StringType())])

In [ ]:
#############################################################################################################
#############################################################################################################
#############################################################################################################
for event in range(len(event_end)):
    start_time = time.time()
    # Window of analysis
    D_Start   = datetime(event_start[event][2],event_start[event][0],event_start[event][1]).date()
    D_End     = datetime(event_end[event][2],event_end[event][0],event_end[event][1]).date()
    CSV_PRINT = YYYYMMDD_string(D_Start) + "_" + YYYYMMDD_string(D_End) + "_" + Variable + ".csv"
    SHP_PRINT = YYYYMMDD_string(D_Start) + "_" + YYYYMMDD_string(D_End) + "_" + Variable + ".shp" 
    DIR_PRINT = YYYYMMDD_string(D_Start) + "_" + YYYYMMDD_string(D_End) 
    # Creating event director for .shp and .csv files 
    Path      = os.path.join(Parent_dir, DIR_PRINT) 
    try: os.mkdir(Path) 
    except: pass
    # 
    Dates     = []
    D         = D_End-D_Start
    D         = D.days+1
    print("------------------------------------------------------------------------")
    print("Summing Precip. for: ")
    for x in range(D):
        Date_Keep = D_Start+timedelta(days=x)
        Dates.append(Date_Keep) 
        print(Date_Keep)
    print("------------------------------------------------------------------------")   
    Year_list = []
    for days in Dates:
        Year_list.append(days.year)
    Year_list = np.unique(Year_list)
    print("Need yearly .csv files for: ") 
    print(Year_list[:])
    #############################################################################################################
    # Repeat for yearly GHCND data
    List_Archive_Dir = find_files( Archive_Dir, suffix=".csv" )
    print("------------------------------------------------------------------------")
    for year in Year_list:
        csv_file = str(year)+'.csv'
        if csv_file not in List_Archive_Dir or Overwrite==1:
            print("Downloading "+csv_file+'.gz')
            Station_File_URL = GHCND_Base_FTP + 'by_year/' + csv_file + '.gz'
            urllib.request.urlretrieve(Station_File_URL, csv_file + '.gz')
            shutil.move(csv_file + '.gz',Archive_Dir)
            with gzip.open(Archive_Dir+csv_file+'.gz', 'rb') as f_in:
                with open(Archive_Dir+csv_file, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
        else: print(csv_file+" is Downloaded: "+Archive_Dir)
    #############################################################################################################
    # Parse the station metadata
    #Station_File         = 'ghcnd-inventory.txt'
    #Station_Inventory    = pd.read_csv(Archive_Dir+Station_File, header=None, delimiter=r"\s+")
    #Station_Inventory.rename(columns={0:'GHCND-ID',1:'Lat',2:'Lon',3:'Variable',4:'YYYY_Start',5:'YYYY_End'}, inplace=True)
    #Station_Inventory    = Station_Inventory[Station_Inventory['Variable']==Variable].reset_index(drop=True)
    #Station_Inventory    = Station_Inventory[(Station_Inventory['YYYY_Start']<=D_Start.year) & \
    #                                         (Station_Inventory['YYYY_End']>=D_End.year)].reset_index(drop=True)
    #print("------------------------------------------------------------------------")
    #print(Station_File+" file has been parsed!")
    #############################################################################################################
    # Turn into dataframe
    Station_Metadata_df = pd.DataFrame(Station_Metadata_np)
    Station_Metadata_df.rename(columns={0:'GHCND-ID',1:'Lat',2:'Lon',3:'Boolean'}, inplace=True)
    Station_Metadata_df = Station_Metadata_df[Station_Metadata_df['Boolean']=='1'].reset_index(drop=True)
    #Station_Metadata_df = Station_Metadata_df.merge(Station_Inventory, on='GHCND-ID', how='inner').reset_index(drop=True)
    print("------------------------------------------------------------------------")
    print("There are "+str(len(Station_Metadata_df))+" stations within the analysis")
    print("------------------------------------------------------------------------")
    #############################################################################################################
    #############################################################################################################
    #############################################################################################################

    #Load massive csv file as a spark dataframe
    if len(Year_list)>1:
        Yearly_metadata_00 = spark.read.csv(Archive_Dir+str(Year_list[0])+'.csv', header = 'true', schema=schema)
        Yearly_metadata_01 = spark.read.csv(Archive_Dir+str(Year_list[1])+'.csv', header = 'true', schema=schema)
        Yearly_metadata    = Yearly_metadata_00.union(Yearly_metadata_01)
    else: Yearly_metadata = spark.read.csv(Archive_Dir+csv_file, header = 'true', schema=schema)
    print("1) Pyspark dataframe loaded...")
    #FILTER DATA TO MAKE IT MORE MANAGABLE
    Type_drop       = ['TAVG','SNWD','TMAX','TMIN','ACMC','ACMH','ACSC','ACSH','AWND','DAEV','DAPR','DASF','DATN',
                       'DATX','DAWM','DWPR','EVAP','FMTM','FRGB','FRGT','FRTH','GAHT','MDEV','MDPR','MDSF','MDTN',
                       'MDTX','MDWM','MNPN','MXPN','PGTM','PSUN','SN*#','SX*#','THIC','TOBS','TSUN','WDF1','WDF2',
                       'WDF5','WDFG','WDFI','WDFM','WDMV','WESD','WSF1','WSF2','WSF5','WSFG','WSFI','WSFM','WT**',
                       'WVxx','WT01','WT02','WT11','WT13'] #Unuseful data
    Quality_drop    = ["D","G","I","K","L","M","N","O","R","S","T","W","X","Z"] #Fauly flags
    Dates_drop      = []
    for days in Dates:
        YYYY_str    = str(days.year)
        if days.month>9: MM_str = str(days.month)
        else:            MM_str = '0'+str(days.month)
        if days.day>9:   DD_str = str(days.day)
        else:            DD_str = '0'+str(days.day)
        Dates_drop.append(YYYY_str+MM_str+DD_str)
    #REMOVING OBSERVATIONS
    Yearly_metadata = Yearly_metadata[~Yearly_metadata['Type'].isin(Type_drop)] #REMOVE UNWANTED OBS
    Yearly_metadata = Yearly_metadata[Yearly_metadata['YYYYMMDD'].isin(Dates_drop)] #REMOVE DATES THAT AREN'T IN TIME-WINDOW
    Yearly_metadata = Yearly_metadata[Yearly_metadata['VALUE'] != 0]  #REMOVE 0 OBS
    #STATIONS W/FLAGGED DAILY TOTALS
    Flagged_Tots    = Yearly_metadata[~Yearly_metadata['QFLAG1'].isin(Quality_drop)==False]
    Flagged_Stat    = Flagged_Tots.select("Station").distinct()
    Flagged_array   = [row.Station for row in Flagged_Stat.collect()]
    #ROMVING SUSPECT STATIONS
    Yearly_metadata = Yearly_metadata[~Yearly_metadata['Station'].isin(Flagged_array)]
    print("2) Pyspark dataframe cleaned...")
    #SEPERATING DATA BY WEATHER OBSERVATION
    Snowfall             = Yearly_metadata[Yearly_metadata['Type']=='SNOW']
    #SnowWaterEq          = Yearly_metadata[Yearly_metadata['Type']=='WESF']
    #LiquidWaterEq        = Yearly_metadata[Yearly_metadata['Type']=='PRCP']
    print("3) Snowfall, SWE, and LWE dataframes created...")
    #SUM BY STATION
    Snowfall_amnt        = Snowfall.groupBy('Station').sum()
    Snowfall_amnt        = Snowfall_amnt[Snowfall_amnt['sum(VALUE)']>Snowfall_TH].select("*").toPandas()
    #SnowWaterEq_amnt     = SnowWaterEq.groupBy('Station').sum()
    #SnowWaterEq_amnt     = SnowWaterEq_amnt[SnowWaterEq_amnt['sum(VALUE)']>SnowWaterEq_TH].select("*").toPandas()
    #LiquidWaterEq_amnt   = LiquidWaterEq.groupBy('Station').sum()
    #LiquidWaterEq_amnt   = LiquidWaterEq_amnt[LiquidWaterEq_amnt['sum(VALUE)']>SnowWaterEq_TH].select("*").toPandas()
    print("4) Snowfall, SWE, and LWE dataframes summed by dates...")
    #JOIN COORDINATES 
    Snowfall_amnt        = pd.merge(Station_Metadata_df, Snowfall_amnt, left_on='GHCND-ID',right_on='Station',how='inner') #JOIN COORDS
    #SnowWaterEq_amnt     = pd.merge(Station_Metadata_df, SnowWaterEq_amnt, left_on='GHCND-ID',right_on='Station',how='inner') #JOIN COORDS
    #LiquidWaterEq_amnt   = pd.merge(Station_Metadata_df, LiquidWaterEq_amnt, left_on='GHCND-ID',right_on='Station',how='inner') #JOIN COORDS
    print("5) Snowfall, SWE, and LWE dataframes joined by station...")

    #############################################################################################################
    #############################################################################################################
    #############################################################################################################
    lons = np.array(Snowfall_amnt['Lon']).astype(float)
    lats = np.array(Snowfall_amnt['Lat']).astype(float)
    data = np.array(Snowfall_amnt['sum(VALUE)']).astype(float)*0.0393701
    
    save_data     = np.concatenate((np.expand_dims(np.asarray(lons),1), np.expand_dims(np.asarray(lats),1), 
                                     np.expand_dims(np.asarray(data),1)),axis=1)
    save_data     = pd.DataFrame(save_data)
    save_data.to_csv("complete_list"+"_"+CSV_PRINT, sep=',',index=False)

In [ ]:
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    from matplotlib.backends.backend_pdf import PdfPages
    from matplotlib.patches import Path, PathPatch
    from matplotlib.colors import BoundaryNorm
    from cartopy.io.shapereader import Reader
    from pykrige.ok import OrdinaryKriging
    import matplotlib.colors as mcolors
    from matplotlib.lines import Line2D
    import matplotlib.ticker as mticker
    import cartopy.feature as cfeature
    import pykrige.kriging_tools as kt
    import matplotlib.pyplot as plt
    from itertools import compress
    import cartopy.crs as ccrs
    import cartopy.crs as ccrs
    import matplotlib as mpl
    from os import listdir
    import pandas as pd
    import numpy as np
    import math
    import os
    # Colorbar function -------------------------------------------------------------------------------
    def make_colorbar(ax, mappable, **kwargs):
        divider = make_axes_locatable(ax)
        orientation = kwargs.pop('orientation', 'vertical')
        if orientation == 'vertical':
            loc = 'right'
        elif orientation == 'horizontal':
            loc = 'bottom'
        cax = divider.append_axes(loc, '5%', pad='3%', axes_class=mpl.pyplot.Axes)
        ax.get_figure().colorbar(mappable, cax=cax, orientation=orientation,label="inch", 
                                     ticks=[0,0.1,1,2,3,4,6,8,12,18,24,30,36,48,60,72,96])
    # Creating colorbar
    clevs     = [0,0.1,1,2,3,4,6,8,12,18,24,30,36,48,60,72,96]
    cmap_data = [(0.96456693, 0.96456693, 0.96456693),
                     (0.77952756, 0.86614173, 0.92125984),
                     (0.51574803, 0.73228346, 0.86220472),
                     (0.32283465, 0.58661417, 0.77952756),
                     (0.18897638, 0.42913386, 0.66535433),
                     (0.17716535, 0.28740157, 0.63385827),
                     (1.        , 1.        , 0.6496063 ),
                     (1.        , 0.80314961, 0.16535433),
                     (1.        , 0.6023622 , 0.16141732),
                     (0.88188976, 0.21259843, 0.15748031),
                     (0.68503937, 0.15354331, 0.16929134),
                     (0.50787402, 0.16535433, 0.16535433),
                     (0.33858268, 0.16535433, 0.16535433),
                     (0.83070866, 0.83070866, 1.        ),
                     (0.68110236, 0.62204724, 0.87007874),
                     (0.57086614, 0.43307087, 0.7007874 )]
    cmap_snow      = mcolors.ListedColormap(cmap_data, 'precipitation')
    norm_snow      = mcolors.BoundaryNorm(clevs, cmap_snow.N)
    # Figure
    fig = plt.figure(figsize=(10, 10))
    # Map Resources
    ax  = plt.subplot(1,1,1, projection=ccrs.Mercator())
    ax.set_title('Historical Snowfall Interpolation: '+'\n'+'title_str', fontsize=24, fontweight='bold')
    ax.set_extent([West, East, South, North])
    #ax.add_geometries(Reader(fname_can).geometries(), ccrs.PlateCarree(), facecolor="w", hatch='\\\\\\\\',  edgecolor='black', lw=0.7)
    #ax.add_geometries(Reader(fname_mask).geometries(), ccrs.PlateCarree(), facecolor="w",  edgecolor='none')
    # Non-outliers
    lon  = XYZ_df[(XYZ_df['Boolean']==1) & (XYZ_df['data']>0)]['Lon']
    lat  = XYZ_df[(XYZ_df['Boolean']==1) & (XYZ_df['data']>0)]['Lat']
    data = XYZ_df[(XYZ_df['Boolean']==1) & (XYZ_df['data']>0)]['data']
    cb  = plt.scatter(lon, lat,
                     c=data, s=10, cmap=cmap_snow, norm=norm_snow,
                     alpha=1, edgecolors='k', linewidths=0.5,
                         zorder=4,marker='o', transform=ccrs.PlateCarree())
    # Outliers
    lon  = XYZ_df[(XYZ_df['Boolean']==0) & (XYZ_df['data']>0)]['Lon']
    lat  = XYZ_df[(XYZ_df['Boolean']==0) & (XYZ_df['data']>0)]['Lat']
    data = XYZ_df[(XYZ_df['Boolean']==0) & (XYZ_df['data']>0)]['data']
    cb  = plt.scatter(lon, lat,
                     c=data, s=15, cmap=cmap_snow, norm=norm_snow,
                     alpha=1, edgecolors='k', linewidths=0.5,
                         zorder=5,marker='X', transform=ccrs.PlateCarree())
    
    # Dumby Points
    lon  = XYZ_df[(XYZ_df['Boolean']==1) & (XYZ_df['data']==0)]['Lon']
    lat  = XYZ_df[(XYZ_df['Boolean']==1) & (XYZ_df['data']==0)]['Lat']
    plt.scatter(lon, lat, s=5,
                     alpha=1,facecolor='k', edgecolors='k', linewidths=0.5,
                         zorder=4,marker='o', transform=ccrs.PlateCarree())
    
    ax.add_feature(cfeature.STATES, facecolor="none", edgecolor='black', lw=0.5)
    ax.add_feature(cfeature.LAKES, facecolor="lightcyan", edgecolor='black', lw=0.5)
    # Colorbar
    make_colorbar(ax, cb, pad=0)
    pp = PdfPages('Testing.pdf')
    pp.savefig(fig)
    pp.close()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.patches import Path, PathPatch
from matplotlib.colors import BoundaryNorm
from cartopy.io.shapereader import Reader
from pykrige.ok import OrdinaryKriging
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import pykrige.kriging_tools as kt
import matplotlib.pyplot as plt
from itertools import compress
import cartopy.crs as ccrs
import cartopy.crs as ccrs
import matplotlib as mpl
from os import listdir
import pandas as pd
import numpy as np
import math
import os

#####################################################################################################################
### SETTING CONSTANTS
#####################################################################################################################
# Plotting window
West_NE    = -82
East_NE    = -66
South_NE   = 37
North_NE   = 48
dxdy       = 0.5
dxkrig     = 0.1
Buffer_E   = 2
Buffer_W   = 10
Buffer_S   = 10
Buffer_N   = 2 

# Shapefiles for cartopy
shp_path      = 'C:/Users/Mike/Desktop/Graduate_Work/Evaluation_Procedure/NWP_Domains_shp/Boundaries_Shapefiles/'
fname_usa     = shp_path+'United_States_States_5m/cb_2016_us_state_5m'
fname_can     = shp_path+'Canada_trsmd/canada_tr'
fname_usa_cnt = shp_path+'/cb_2018_us_county_5m/cb_2018_us_county_5m.shp'
fname_neas_cl = shp_path+'/Northeast_Selection/Northeast_Outline.shp'
        
# Creating colorbar
clevs     = [0,0.1,1,2,3,4,6,8,12,18,24,30,36,48,60,72,96]
cmap_data = [(0.96456693, 0.96456693, 0.96456693),
                 (0.77952756, 0.86614173, 0.92125984),
                 (0.51574803, 0.73228346, 0.86220472),
                 (0.32283465, 0.58661417, 0.77952756),
                 (0.18897638, 0.42913386, 0.66535433),
                 (0.17716535, 0.28740157, 0.63385827),
                 (1.        , 1.        , 0.6496063 ),
                 (1.        , 0.80314961, 0.16535433),
                 (1.        , 0.6023622 , 0.16141732),
                 (0.88188976, 0.21259843, 0.15748031),
                 (0.68503937, 0.15354331, 0.16929134),
                 (0.50787402, 0.16535433, 0.16535433),
                 (0.33858268, 0.16535433, 0.16535433),
                 (0.83070866, 0.83070866, 1.        ),
                 (0.68110236, 0.62204724, 0.87007874),
                 (0.57086614, 0.43307087, 0.7007874 )]
cmap_snow      = mcolors.ListedColormap(cmap_data, 'precipitation')
norm_snow      = mcolors.BoundaryNorm(clevs, cmap_snow.N)
#####################################################################################################################
### DEFINING FUNCTIONS
#####################################################################################################################
# Colorbar function -------------------------------------------------------------------------------
def make_colorbar(ax, mappable, **kwargs):
    divider = make_axes_locatable(ax)
    orientation = kwargs.pop('orientation', 'vertical')
    if orientation == 'vertical':
        loc = 'right'
    elif orientation == 'horizontal':
        loc = 'bottom'
    cax = divider.append_axes(loc, '5%', pad='3%', axes_class=mpl.pyplot.Axes)
    ax.get_figure().colorbar(mappable, cax=cax, orientation=orientation,label="inch", 
                                 ticks=[0,0.1,1,2,3,4,6,8,12,18,24,30,36,48,60,72,96])
# Spatial Outlier Algorithm -------------------------------------------------------------------------------
def Outlier_Check(XYZ,dz,pwr):
    dsize    = XYZ.shape
    n        = int((dsize[0]))
    print("All available observations pre-outlier analysis: " + str(n))
    #Pre-alo Variables
    IDW      = np.zeros((n,n))
    widw     = np.zeros((n,n))
    hij      = np.zeros((n,n))
    #Loop through each point
    for i in range(n):
        #Determine euclidean distance (km) from each observation [RadEarth*c/1000]
        LonD        = np.radians(XYZ[:,0])
        LatD        = np.radians(XYZ[:,1])
        delta_phi   = LatD[i]-LatD
        delta_lamda = LonD[i]-LonD
        a           = np.sin(delta_phi/2)**2 + np.cos(LatD[i])*np.cos(LatD)*np.sin(delta_lamda/2.0)**2
        a[a==0]     = np.nan
        c           = 2*np.arctan(np.sqrt(a)/np.sqrt(1-a)) #getting errors in a
        #Replace these values with nan... avoid error message
        c[c==0]     = np.nan
        #Determine Inverse Distance Weight between each observation
        IDW[i,:]    = 1/(RadEarth*c/1000)**pwr
        IDW[i,i]    = np.nan
        #Index observations for each stations
        widw[:,i]   = XYZ[i,2]
        widw[i,i]   = np.nan
        #Determine Euclidean distance b/n each station
        hij[i,:]    = RadEarth*c/1000
        hij[i,i]    = 99999
    #Are the observations w/n threshold that has been defined
    Eucl_Logical    = hij<dz #getting errors in hij
    Sum_row_Eucl    = np.nansum(IDW*Eucl_Logical, axis=1)
    #Weighted distance b/n observations as a ratio
    Wij = np.full((n, n),np.nan)
    for i in range(n):
        if Sum_row_Eucl[i] > 0:
            Wij[i,:] = np.round(IDW[i,:]/Sum_row_Eucl[i],2)        
        Wij[i,i] = np.nan
    #Observations with no cooresponding stations within a given threshold will have Wij == inf... 
    #Replace these values with nan
    std_idw           = np.std(XYZ[:,2])
    weighted_variable = np.nansum(Wij*widw*Eucl_Logical,axis=1)
    #Z-score for a normal distribution
    Zscore_i          = (XYZ[:,2]-weighted_variable)/std_idw
    non_outliers      = abs(Zscore_i)<1.960
    #Return outliers
    XYZ_outliers      = XYZ[non_outliers==False]
    #Return nonoutliers
    XYZ_nonoutliers   = XYZ[non_outliers==True]
    print("Outliers detected: " + str(len(XYZ_outliers)))
    print("Specifications: Interpolation == IDW to the " + str(pwr) + " power")
    print("Specifications: Search Radius == " + str(dz) + "km")
    print("----------------------------------------------")
    return(XYZ_nonoutliers,XYZ_outliers)
# Adding '0' inch observations function -------------------------------------------------------------------------------
def Dumby_Check(lons, lats, query_lon, query_lat, dz):
    hij        = np.zeros((len(lons),1))
    for z in range(len(lons)):
        # Constant
        R = RadEarth
        # Observation points
        lat1 = lats[z]
        lon1 = lons[z]
        # Query/grid point
        lat2 = query_lat
        lon2 = query_lon
        # Calculation
        phi1, phi2 = math.radians(lat1), math.radians(lat2) 
        dphi       = math.radians(lat2 - lat1)
        dlambda    = math.radians(lon2 - lon1)
        a          = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
        # Euclidean distance in km
        hij[z,0]   = ((2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a)))/1000)
    hij            = np.sum(hij<=dz)
    # Return the number of stations within dz kms of an observation.
    return(hij)
# Points for interpolation check -------------------------------------------------------------------------------
def Density_Check(lons,lats,data,dz,dp):
    XYZ      = np.concatenate((np.expand_dims(lons,1), np.expand_dims(lats,1), np.expand_dims(data,1)), axis=1)
    print("----------------------------------------------")
    print("Initial number of observations: " + str(len(XYZ)))
    n        = len(XYZ)
    #Pre-alo Variables
    hij      = np.zeros((n,n))
    #Loop through each point
    for i in range(n):
        #Determine euclidean distance (km) from each observation [RadEarth*c/1000]
        LonD        = np.radians(XYZ[:,0])
        LatD        = np.radians(XYZ[:,1])
        delta_phi   = LatD[i]-LatD
        delta_lamda = LonD[i]-LonD
        a           = np.sin(delta_phi/2)**2 + np.cos(LatD[i])*np.cos(LatD)*np.sin(delta_lamda/2.0)**2
        a[a==0]     = np.nan
        c           = 2*np.arctan(np.sqrt(a)/np.sqrt(1-a)) #getting errors in a
        #Replace these values with nan... avoid error message
        c[c==0]     = np.nan
        #Determine Euclidean distance b/n each station
        hij[i,:]    = RadEarth*c/1000
        hij[i,i]    = 99999
    #Are the observations w/n threshold that has been defined
    Eucl_Logical    = hij<dz #getting errors in hij
    Eucl_Logical    = np.expand_dims(np.sum(Eucl_Logical,axis=1),1)
    XYZ             = np.asarray(list(compress(XYZ,(Eucl_Logical>dp)==True)))
    print("Clustered number of observations: " + str(int(len(XYZ))))
    print("Specifications: Distance == " + str(dz) + "km Point Density == " + str(dp))
    print("----------------------------------------------")
    return(XYZ)
#####################################################################################################################
### LOAD AND PRE-PROCESS DATA, AND INTERPOLATE THE DATA
#####################################################################################################################
Path_Files = 'C:/Users/Mike/Desktop/GHCND/attempt_01/complete_list/'
File_List  = find_files(Path_Files, suffix=".csv" )
for z in range(1):
    # Load and pre-process
    Testing    = pd.read_csv(Path_Files+File_List[-1])
    # Limit area for computation efficiency
    Region_Log = np.empty([len(Testing),1], dtype=object)
    for x in range(len(Testing)):
        if (float(Testing['1'][x]) < North_NE+Buffer_N) and (float(Testing['1'][x]) > South_NE-Buffer_S) and \
           (float(Testing['0'][x]) <  East_NE+Buffer_E) and (float(Testing['0'][x]) >  West_NE-Buffer_W): Region_Log[x,0] = '1'
        else: Region_Log[x,0] = '0'
    Testing = Testing[Region_Log=='1'].reset_index()
    
    # Creating some names
    File       = File_List[z]
    File       = File.replace('complete_list_', '')
    File_title = File.replace('_SNOW.csv', '')
    File_png   = File.replace('_SNOW.csv', '.png')
    File_pdf   = File.replace('_SNOW.csv', '.pdf')
    title_str  = File_title[0:4]+"-"+File_title[4:6]+"-"+File_title[6:8]+" to "+File_title[9:13]+"-"+File_title[13:15]+"-"+File_title[15:17]

    # Converting data 
    lons       = Testing['0'].astype(float)
    lats       = Testing['1'].astype(float)
    data       = Testing['2'].astype(float)
    save_data  = np.concatenate((np.expand_dims(np.asarray(lons),1), np.expand_dims(np.asarray(lats),1), 
                                         np.expand_dims(np.asarray(data),1)),axis=1)
    # Determine outliers
    XYZ_nonoutliers,XYZ_outliers  = Outlier_Check(save_data,50,2)
    XYZ_outliers_df               = pd.DataFrame(XYZ_outliers)
    XYZ_outliers_df.to_csv("outliers"+"_"+File, sep=',',index=False)
    # Check density
    XYZ                                    = Density_Check(XYZ_nonoutliers[:,0],XYZ_nonoutliers[:,1],XYZ_nonoutliers[:,2],50,2) 
    # Begin to set grid for Kriging interpolation
    grid_lon       = np.arange(np.amin(XYZ[:,0]), np.amax(XYZ[:,0]), 0.5) #grid_space is the desired delta/step of the output array 
    grid_lat       = np.arange(np.amin(XYZ[:,1]), np.amax(XYZ[:,1]), 0.5) #dxdy
    xintrp, yintrp = np.meshgrid(grid_lon, grid_lat)
    # Add dumby points [obs == 0 inch]
    dumby_lon      = []
    dumby_lat      = []
    n_iterations   = xintrp.shape
    for x in range(n_iterations[0]):
        for y in range(n_iterations[1]):
            query_log = Dumby_Check(XYZ[:,0], XYZ[:,1], xintrp[x,y], yintrp[x,y], 50)
            if query_log==0: 
                dumby_lon.append(xintrp[x,y])
                dumby_lat.append(yintrp[x,y])
    dumby_data                             = np.concatenate((np.expand_dims(np.asarray(dumby_lon),1), 
                                                             np.expand_dims(np.asarray(dumby_lat),1), 
                                                             np.zeros((len(dumby_lat),1))),axis=1)
    XYZ_nonoutliers                        = np.concatenate((dumby_data,XYZ_nonoutliers),axis=0)
    XYZ_nonoutliers_df                        = pd.DataFrame(XYZ_outliers)
    XYZ_nonoutliers_df.to_csv("nonoutliers"+"_"+File, sep=',',index=False)
    XYZ_nonoutliers = XYZ_nonoutliers[XYZ_nonoutliers[:,2]!=0]
    Region_Log      = np.empty([len(XYZ_nonoutliers),1], dtype=object)
    for i in range(len(XYZ_nonoutliers)):
        if (XYZ_nonoutliers[i,1] < North_NE+Buffer_N) and (XYZ_nonoutliers[i,1] > South_NE-Buffer_S) and \
           (XYZ_nonoutliers[i,0] <  East_NE+Buffer_E) and (XYZ_nonoutliers[i,0] >  West_NE-Buffer_W): Region_Log[i,0] = '1'
        else: Region_Log[i,0] = '0'
    XYZ_nonoutliers = XYZ_nonoutliers[Region_Log[:,0]=='1']
    # Nonoutlier data including 0 dumby points
    Lon   = XYZ_nonoutliers[:,0].astype(float)
    Lat   = XYZ_nonoutliers[:,1].astype(float)
    data  = XYZ_nonoutliers[:,2].astype(float)
    # Outlier data 
    Lon_o   = XYZ_outliers[:,0].astype(float)
    Lat_o   = XYZ_outliers[:,1].astype(float)
    data_o  = XYZ_outliers[:,2].astype(float)
    #####################################################################################################################
    ### SPHERICAL KRIGIN INTERPOLATION ONTO GRID
    #####################################################################################################################
    print('Ord. Kriging #1')
    grid_lon   = np.arange(np.amin(Lon), np.amax(Lon), 0.1)#dxkrig
    grid_lat   = np.arange(np.amin(Lat), np.amax(Lat), 0.1)
    North      = np.amax(Lat)
    East       = np.amax(Lon)
    South      = np.amin(Lat)
    West       = np.amin(Lon)
    print('Ord. Kriging #2')
    OK      = OrdinaryKriging(Lon, Lat, data, variogram_model='spherical', verbose=True,
                                  enable_plotting=False, coordinates_type='geographic')
    print('Ord. Kriging #3')
    z1, ss1 = OK.execute('grid', grid_lon, grid_lat)
    print('Ord. Kriging #4')
    xintrp, yintrp = np.meshgrid(grid_lon, grid_lat)
    print('Ord. Kriging #5')
    z1[z1<0] = 0 #change negative values to 0
    #####################################################################################################################
    ### PLOTTING
    #####################################################################################################################
    # Figure
    fig = plt.figure(figsize=(10, 10))

    # Map Resources
    ax  = plt.subplot(1,1,1, projection=ccrs.Mercator())
    ax.set_title('Historical Snowfall Interpolation: '+'\n'+title_str, fontsize=24, fontweight='bold')
    ax.set_extent([West_NE, East_NE, South_NE, North_NE])
    #ax.add_feature(cfeature.OCEAN, facecolor="lightcyan",  edgecolor=None, zorder=1)
    #ax.set_facecolor('xkcd:salmon')
    #ax.add_feature(cfeature.LAKES, facecolor="lightcyan",  edgecolor=None, zorder=1)
    #ax.add_geometries(Reader(fname_usa).geometries(), ccrs.PlateCarree(), facecolor="none", edgecolor='black', lw=0.7,zorder=3)
    #ax.add_geometries(Reader(fname_usa_cnt).geometries(), ccrs.PlateCarree(), facecolor="none", edgecolor='lightgray', lw=0.5,zorder=2)
    #ax.add_geometries(Reader(fname_can).geometries(), ccrs.PlateCarree(), facecolor="w", hatch='\\\\\\\\',  edgecolor='black', lw=0.7,zorder=1)
    ax.add_geometries(Reader(fname_neas_cl).geometries(), ccrs.PlateCarree(), facecolor="none",  edgecolor='black', lw=0.7,zorder=3)
    
    # Kriging Interpolation Plot
    cb = plt.contourf(xintrp, yintrp, z1, np.asarray(clevs), cmap=cmap_snow, norm=norm_snow, vmin = 0, vmax = 96, transform=ccrs.PlateCarree())

    # Non-outliers Observations
    cb = plt.scatter(Lon[data!=0], Lat[data!=0], c=data[data!=0], s=10, cmap=cmap_snow, norm=norm_snow, alpha=1, edgecolors='k', linewidths=0.5,
                         zorder=4,marker='o', transform=ccrs.PlateCarree())
    # Outliers Observations
    cb = plt.scatter(Lon_o, Lat_o, c=data_o, s=50, cmap=cmap_snow, norm=norm_snow, alpha=1, edgecolors='k', linewidths=0.5,
                         zorder=4,marker='X', transform=ccrs.PlateCarree())
    # Contours 
    CS = ax.contour(xintrp, yintrp, z1, [12,24,36,48],linestyles='dashed', colors='k',
                        transform=ccrs.PlateCarree())  # Negative contours default to dashed.
    ax.clabel(CS, fontsize=16,inline=True, fmt='%1.0f')

    # Colorbar
    make_colorbar(ax, cb, pad=0)

    # Graticule
    #gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
    #                      linewidth=0.75, color='gray', alpha=0.8, linestyle='--')
    #gl.ylocator = mticker.FixedLocator([22, 28, 32, 36, 40, 44, 48, 52])
    #gl.xlocator = mticker.FixedLocator([-106, -100, -94, -88, -82, -76, -70, -64, -58, -52])
    #gl.xlabels_top = False
    #gl.ylabels_right = False
    #gl.xlabel_style = {'size': 15, 'color': 'gray'}
    #gl.ylabel_style = {'size': 15, 'color': 'gray'}

    # Saving
    #fig.savefig(File_png)
    #pp = PdfPages(File_pdf)
    #pp.savefig(fig)
    #pp.close()

In [3]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.patches import Path, PathPatch
from matplotlib.colors import BoundaryNorm
from cartopy.io.shapereader import Reader
import cartopy.io.shapereader as shpreader
from shapely.geometry import MultiLineString
from pykrige.ok import OrdinaryKriging
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import pykrige.kriging_tools as kt
import matplotlib.pyplot as plt
from itertools import compress
import cartopy.crs as ccrs
import cartopy.crs as ccrs
import matplotlib as mpl
from os import listdir
import pandas as pd
import numpy as np
import time
import math
import os

from datetime import datetime

LIST_DIR = pd.read_csv('C:/Users/Mike/NESIS_Kriging/Final_Interpolations/Listing.csv')
TEXT_DIR = 'C:/Users/Mike/NESIS_Kriging/Final_Interpolations/grid_txt/'

#####################################################################################################################
### SETTING CONSTANTS
#####################################################################################################################
# Plotting window
West_NE    = -82
East_NE    = -66
South_NE   = 37
North_NE   = 48
dxdy       = 0.5
dxkrig     = 0.1
Buffer_E   = 2
Buffer_W   = 10
Buffer_S   = 10
Buffer_N   = 2 

# Shapefiles for cartopy
shp_path      = 'C:/Users/Mike/Desktop/Graduate_Work/Evaluation_Procedure/NWP_Domains_shp/Boundaries_Shapefiles/'
fname_usa     = shp_path+'United_States_States_5m/cb_2016_us_state_5m'
fname_can     = shp_path+'Canada_trsmd/canada_tr'
fname_usa_cnt = shp_path+'/cb_2018_us_county_5m/cb_2018_us_county_5m.shp'
fname_neas_cl = shp_path+'/Northeast_Selection/Northeast_Outline.shp'
fname_roads   = shp_path+'/tl_2016_us_primaryroads/tl_2016_us_primaryroads.shp'

ocean = cfeature.NaturalEarthFeature(category='physical', name='ocean',
                                     scale='50m',
                                     facecolor=cfeature.COLORS['water'])

Lakes = cfeature.NaturalEarthFeature(category='physical', name='lakes',
                                     scale='10m',
                                     facecolor='lightcyan')
        
# Creating colorbar
clevs     = [0,0.1,1,2,3,4,6,8,12,18,24,30,36,48,60,72,96]
cmap_data = [(0.96456693, 0.96456693, 0.96456693),
                 (0.77952756, 0.86614173, 0.92125984),
                 (0.51574803, 0.73228346, 0.86220472),
                 (0.32283465, 0.58661417, 0.77952756),
                 (0.18897638, 0.42913386, 0.66535433),
                 (0.17716535, 0.28740157, 0.63385827),
                 (1.        , 1.        , 0.6496063 ),
                 (1.        , 0.80314961, 0.16535433),
                 (1.        , 0.6023622 , 0.16141732),
                 (0.88188976, 0.21259843, 0.15748031),
                 (0.68503937, 0.15354331, 0.16929134),
                 (0.50787402, 0.16535433, 0.16535433),
                 (0.33858268, 0.16535433, 0.16535433),
                 (0.83070866, 0.83070866, 1.        ),
                 (0.68110236, 0.62204724, 0.87007874),
                 (0.57086614, 0.43307087, 0.7007874 )]
cmap_snow      = mcolors.ListedColormap(cmap_data, 'precipitation')
norm_snow      = mcolors.BoundaryNorm(clevs, cmap_snow.N)
#####################################################################################################################
### DEFINING FUNCTIONS
#####################################################################################################################
# Colorbar function -------------------------------------------------------------------------------
def make_colorbar(ax, mappable, **kwargs):
    divider = make_axes_locatable(ax)
    orientation = kwargs.pop('orientation', 'vertical')
    if orientation == 'vertical':
        loc = 'right'
    elif orientation == 'horizontal':
        loc = 'bottom'
    cax = divider.append_axes(loc, '5%', pad='3%', axes_class=mpl.pyplot.Axes)
    ax.get_figure().colorbar(mappable, cax=cax, orientation=orientation,label="inch",
                                 ticks=[0,0.1,1,2,3,4,6,8,12,18,24,30,36,48,60,72,96])

from datetime import datetime
def Title_STR(dt_start,dt_end):
    START  = dt_start.split("/")
    START  = datetime(int(START[2]),int(START[0]),int(START[1]))
    START  = START.strftime("%B %d, %Y,")
    END    = dt_end.split("/")
    END    = datetime(int(END[2]),int(END[0]),int(END[1]))
    END    = END.strftime("%B %d, %Y")
    STRING = START+" to "+END
    return(STRING)

RadEarth   = 6371000 #m Euclidean Dist.
def Outlier_Check(XYZ,dz,pwr):
    dsize    = XYZ.shape
    n        = int((dsize[0]))
    print("All available observations pre-outlier analysis: " + str(n))
    #Pre-alo Variables
    IDW      = np.zeros((n,n))
    widw     = np.zeros((n,n))
    hij      = np.zeros((n,n))
    #Loop through each point
    for i in range(n):
        #Determine euclidean distance (km) from each observation [RadEarth*c/1000]
        LonD        = np.radians(XYZ[:,0])
        LatD        = np.radians(XYZ[:,1])
        delta_phi   = LatD[i]-LatD
        delta_lamda = LonD[i]-LonD
        a           = np.sin(delta_phi/2)**2 + np.cos(LatD[i])*np.cos(LatD)*np.sin(delta_lamda/2.0)**2
        a[a==0]     = np.nan
        c           = 2*np.arctan(np.sqrt(a)/np.sqrt(1-a)) #getting errors in a
        #Replace these values with nan... avoid error message
        c[c==0]     = np.nan
        #Determine Inverse Distance Weight between each observation
        IDW[i,:]    = 1/(RadEarth*c/1000)**pwr
        IDW[i,i]    = np.nan
        #Index observations for each stations
        widw[:,i]   = XYZ[i,2]
        widw[i,i]   = np.nan
        #Determine Euclidean distance b/n each station
        hij[i,:]    = RadEarth*c/1000
        hij[i,i]    = 99999
    #Are the observations w/n threshold that has been defined
    Eucl_Logical    = hij<dz #getting errors in hij
    Sum_row_Eucl    = np.nansum(IDW*Eucl_Logical, axis=1)
    #Weighted distance b/n observations as a ratio
    Wij = np.full((n, n),np.nan)
    for i in range(n):
        if Sum_row_Eucl[i] > 0:
            Wij[i,:] = np.round(IDW[i,:]/Sum_row_Eucl[i],2)
        Wij[i,i] = np.nan
    #Observations with no cooresponding stations within a given threshold will have Wij == inf...
    #Replace these values with nan
    std_idw           = np.std(XYZ[:,2])
    weighted_variable = np.nansum(Wij*widw*Eucl_Logical,axis=1)
    #Z-score for a normal distribution
    Zscore_i          = (XYZ[:,2]-weighted_variable)/std_idw
    Logical_array     = [np.logical_and(Zscore_i > -1.280, Zscore_i < 4)]
    #non_outliers      = abs(Zscore_i)<1.960
    #Return outliers
    XYZ_outliers      = XYZ[Logical_array[0]==False]
    #Return nonoutliers
    XYZ_nonoutliers   = XYZ[Logical_array[0]==True]
    #print("Outliers detected: " + str(len(XYZ_outliers)))
    #print("Specifications: Interpolation == IDW to the " + str(pwr) + " power")
    #print("Specifications: Search Radius == " + str(dz) + "km")
    #print("----------------------------------------------")
    return(XYZ_nonoutliers,XYZ_outliers,Zscore_i)

Kriging_Grid = 0.025
West_NE      = -82
East_NE      = -66
South_NE     = 37
North_NE     = 48
#####################################################################################################################
### PLOTTING
#####################################################################################################################
Kriging_Grid = 0.025
grid_lon     = np.arange(West_NE , East_NE , Kriging_Grid)
grid_lat     = np.arange(South_NE, North_NE, Kriging_Grid)
xintrp, yintrp = np.meshgrid(grid_lon, grid_lat)

for i in range(82,83): #80
    fig = plt.figure(figsize=(10, 8))
    print(LIST_DIR['Grid File'][i])
    print(i)
    now       = datetime.now()
    GRID      = 'C:/Users/Mike/NESIS_Kriging/Final_Interpolations/grid_txt/'+LIST_DIR['Folder'][i]+'/'+LIST_DIR['Grid File'][i]
    ascii_grid= np.loadtxt(GRID, skiprows=0)
    OBS  = 'C:/Users/Mike/NESIS_Kriging/Final_Interpolations/grid_obs/'+LIST_DIR['OBS File'][i]
    XYZ  = pd.read_csv(OBS,skiprows=[0],names=['Lon','Lat','Snowfall_inch'])
    XYZ  = np.asarray(XYZ).astype(float)
    #OUTLIERS
    XYZ_obs        = []
    XYZ_out        = []
    do             = len(XYZ)
    doutlier       = [np.nan,np.nan] #start
    i              = 0
    while (doutlier[-1]-doutlier[-2])!= 0:
        i                         = i+1
        XYZ,XYZ_outliers,Zscore_i = Outlier_Check(XYZ,50,2)
        print(len(XYZ))
        doutlier.append(do-len(XYZ))
        XYZ_obs.append(XYZ)
        XYZ_out.append(XYZ_outliers)
        print("Snow Iteration: "+str(i)+" Total Outliers: "+str(doutlier[-1]))
        if i>10: break
    XYZ_obs = np.unique(np.vstack(XYZ_obs),axis=0)
    XYZ_out = np.unique(np.vstack(XYZ_out),axis=0)
    print("Number of Observations Post-Outliers: "+str(len(XYZ_obs)))
    print("Number of Outliers Post-Outliers: "+str(len(XYZ_out)))
    #naming
    test = LIST_DIR['OBS File'][i]
    test = test.replace("complete_list_","map_")
    test = test.replace("_SNOW.csv","")
    # Map Resources
    ax  = plt.subplot(1,1,1, projection=ccrs.Mercator())
    ax.set_extent([West_NE, East_NE, South_NE, North_NE],crs=ccrs.PlateCarree())
    ax.set_title(Title_STR(LIST_DIR['START'][i],LIST_DIR['END'][i]),
                 fontsize=24,loc='left',fontweight='bold',color="midnightblue")
    #SNOWFALL - SHADED (1)
    cb = plt.contourf(xintrp, yintrp, ascii_grid, np.asarray(clevs), cmap=cmap_snow, norm=norm_snow,zorder=1,
                      vmin = 0, vmax = 96, alpha=1,extend=max, transform=ccrs.PlateCarree())
    print("SNOWFALL")
    #USA COUNTIES - POLYGON (2)
    ax.add_geometries(Reader(fname_usa_cnt).geometries(), ccrs.PlateCarree(), facecolor="none",zorder=2,
                      edgecolor='silver', lw=0.25, alpha=0.80)
    print("COUNTIES")
    #ATLANTIC OCEAN - POLYGON (3)
    ax.add_feature(ocean,zorder=5)    #3
    print("ATLANTIC")
    #USA STATES - POLYGON (4)
    ax.add_geometries(Reader(fname_usa).geometries(), ccrs.PlateCarree(),zorder=7, #4
                      facecolor="none", edgecolor='k', lw=1)
    print("USA STATES")
    #SNOW CONTOURS - SEGMENTS (5)
    CS = ax.contour(xintrp, yintrp, ascii_grid,  [12,24,36,48], linestyles='dashed', colors='k', zorder=3, 
                        transform=ccrs.PlateCarree())
    ax.clabel(CS, fontsize=16, inline=True, fmt='%1.0f') 
    print("CONTOURS")
    #LAKES - POLYGON (6)
    ax.add_feature(Lakes,zorder=4) #5
    print("LAKES")
    #CANADA PROVINCES - POLYGON (7)
    ax.add_geometries(Reader(fname_can).geometries(), ccrs.PlateCarree(), facecolor="w", hatch='\\\\\\\\',
                      edgecolor='black', lw=0.7,zorder=6)
    print("CANADA")
    #OBS - POINT (8)
    #XYZ_nonoutliers,XYZ_outliers,Zscore_i = Outlier_Check(XYZ,50,2)
    cb = plt.scatter(XYZ_obs[:,0], XYZ_obs[:,1], c=XYZ_obs[:,2], s=10, 
                     cmap=cmap_snow, norm=norm_snow, zorder=8, 
                     edgecolors='k', linewidths=0.25, marker='o', transform=ccrs.PlateCarree())
    #OUTLIERS - POINT (9)
    cb = plt.scatter(XYZ_out[:,0], XYZ_out[:,1], c=XYZ_out[:,2], s=25, 
                     cmap=cmap_snow, norm=norm_snow, zorder=9,
                     edgecolors='k', linewidths=0.25,marker='X', transform=ccrs.PlateCarree())
    print("OBSERVATIONS")
    #USA ROADS - SEGMENTS (10)
    #shp   = shpreader.Reader(fname_roads)
    #add_s = shp.records()
    #for add in add_s:
    #    if( add.geometry == None):
    #        pass
    #    else:
    #        lines = add.geometry
    #        line_good=[]
    #        for l in lines:
    #            start_pt = list(l.coords)[0]
    #            for i in range(1,len(l.coords)):
    #                end_pt = list(l.coords)[i]
    #                simple_line = (start_pt, end_pt)
    #                line_good.append(simple_line)
    #                start_pt = end_pt
    #        lines = MultiLineString(line_good)
    #        ax.add_geometries([lines], ccrs.PlateCarree(), edgecolor='slategray', facecolor=None, lw=0.75, zorder=10)   
    #print("ROADS")
    #COLORBAR
    make_colorbar(ax, cb, pad=0)
    #ADJUST FIGURE
    plt.subplots_adjust(top = 0.95, bottom = 0.05, right = 1, left = 0, hspace = 0, wspace = 0)
    #ADD FIG TEXT
    fig.text(0.09,0.02,'Kriging Resolution: 0.025$^\circ$x0.025$^\circ$', ha='left')
    dt_string = now.strftime("%Y/%m/%d %H:%M:%S")
    fig.text(0.85,0.02,"Created: "+dt_string, ha='right')
    #SAVE FIGURE
    fig.savefig(test,dpi=300)
    #CLOSE FGURE
    plt.close(fig=None)
    #time.sleep(60*4) # second

Krig_hole-effect_20100208_20100211.txt
82
All available observations pre-outlier analysis: 3991


C:\Users\Mike\Anaconda3\lib\site-packages\ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in less


3925
Snow Iteration: 1 Total Outliers: 66
All available observations pre-outlier analysis: 3925
3914
Snow Iteration: 2 Total Outliers: 77
All available observations pre-outlier analysis: 3914
3913
Snow Iteration: 3 Total Outliers: 78
All available observations pre-outlier analysis: 3913
3913
Snow Iteration: 4 Total Outliers: 78
Number of Observations Post-Outliers: 3923
Number of Outliers Post-Outliers: 78
SNOWFALL
COUNTIES
ATLANTIC
USA STATES
CONTOURS
LAKES
CANADA
OBSERVATIONS


In [ ]:
Kriging_Grid = 0.025
West_NE      = -82
East_NE      = -66
South_NE     = 37
North_NE     = 48

grid_lon       = np.arange(West_NE , East_NE , Kriging_Grid)
grid_lat       = np.arange(South_NE, North_NE, Kriging_Grid)
xintrp, yintrp = np.meshgrid(grid_lon, grid_lat)
density        = np.zeros(xintrp.shape)

Dir = 'C:/Users/Mike/NESIS_Kriging/complete_list/'
LS  = find_files(Dir, suffix=".csv")
for i in range(len(LS)):
    print(i)
    LS_i  = pd.read_csv(Dir+LS[i])
    Lon_i = LS_i['0']
    Lat_i = LS_i['1']
    for j in range(len(LS_i)):
        q_lon_j = abs(Lon_i[j]-xintrp)
        q_lat_j = abs(Lat_i[j]-yintrp)
        q_coord = q_lon_j+q_lat_j
        idx_xy  = np.unravel_index(q_coord.argmin(),q_coord.shape)
        #print(idx_xy)
        density[idx_xy[0],idx_xy[1]] = density[idx_xy[0],idx_xy[1]] +1

In [194]:
event_start = [[10,29,2020],[11,29,2019],[1,18,2019],[11,14,2018],[3,20,2018],[3,11,2018],[3,5,2018],[3,1,2018],[1,3,2018],
                [3,12,2017],[2,9,2017],[11,17,2016],[1,22,2016],[2,14,2015],[2,8,2015],[1,29,2015],[1,25,2015],
                [12,9,2014],[11,26,2014],[2,11,2014],[1,20,2014],[12,30,2013],[12,13,2013],[3,17,2013],[3,3,2013],
                [2,8,2013],[12,28,2012],[12,24,2012],[10,25,2011],[2,24,2011],[2,1,2011],[1,26,2011],[1,9,2011],
                [12,24,2010],[2,21,2010],[2,12,2010],[2,8,2010],[2,4,2010],[12,28,2009],[12,18,2009],[12,7,2009],
                [2,26,2009],[2,22,2009],[12,21,2008],[12,18,2008],[12,14,2007],[11,30,2007],[4,3,2007],[3,16,2007],
                [2,11,2007],[2,10,2006],[2,28,2005],[1,22,2005],[12,4,2003],[2,14,2003],[1,1,2003],[12,23,2002],[12,28,2000],[2,16,2000],[1,24,2000],[1,24,2000],
                [3,12,1999],[1,13,1999],[3,31,1997],[1,8,1997],[4,9,1996],[3,3,1996],[2,1,1996],[1,6,1996],[12,18,1995],
                [2,2,1995],[2,28,1994],[2,22,1994],[1,16,1994],[1,4,1994],[1,1,1994],[3,12,1993],[2,20,1993],[2,14,1993],
                [12,9,1992],[12,27,1990],[2,9,1988],[1,22,1988],[1,5,1988],[12,13,1987],[11,10,1987],[2,22,1987],[1,21,1987],
                [1,8,1987],[1,1,1987],[3,1,1985],[1,29,1985],[2,24,1984],[2,10,1983],[4,4,1982],[1,20,1982],[1,12,1982],
                [2,17,1979],[2,6,1979],[2,4,1978],[1,17,1978],[1,14,1978],[1,11,1978],[3,21,1977],[1,7,1977],[1,8,1974],
                [12,15,1973],[2,16,1972],[11,23,1971],[2,26,1971],[12,31,1970],[12,25,1969],[2,25,1969],[2,22,1969],[2,8,1969],
                [3,20,1967],[2,6,1967],[12,22,1966],[2,23,1966],[1,28,1966],[1,21,1966],[2,18,1964],[1,9,1964],[12,21,1963],
                [3,5,1962],[2,28,1962],[2,13,1962],[2,1,1961],[1,18,1961],[12,10,1960],[2,29,1960],[2,12,1960],[3,12,1959],
                [3,18,1958],[2,12,1958],[12,3,1957],[3,18,1956],[3,14,1956],[3,3,1956],[2,17,1952],[12,13,1951],[11,22,1950],
                [2,11,1950],[1,29,1949],[1,23,1948],[12,25,1947],[2,27,1947],[2,20,1947],[2,15,1946],[12,17,1945],[1,13,1945],
                [12,8,1944],[2,8,1944],[1,25,1943],[3,28,1942],[2,28,1942],[3,7,1941],[2,13,1940],[11,23,1938],[2,10,1936],
                [1,18,1936],[1,21,1935],[2,23,1934],[12,26,1933],[12,16,1932],[3,4,1931],[12,19,1929],[2,20,1929],[2,16,1927],
                [2,3,1926],[1,7,1926],[1,28,1925],[12,31,1924],[4,1,1924],[2,17,1924],[2,3,1923],[1,26,1922],[2,18,1921],[2,4,1920],
                [1,25,1918],[1,21,1918],[1,12,1918],[12,12,1917],[12,6,1917],[3,2,1917],[3,2,1916],[12,10,1915],[4,3,1915],[3,2,1915],
                [1,29,1915],[2,12,1914],[2,16,1910],[2,10,1910],[1,12,1910],[12,23,1909],[3,2,1909],[1,27,1909],[1,10,1909],[2,16,1908],
                [2,3,1908],[1,29,1908],[2,4,1907],[3,17,1906],[3,12,1906],[1,27,1904],[2,14,1903],[12,11,1902],[3,3,1902],[2,13,1902],
                [2,1,1901],[3,15,1900],[2,26,1900]]
event_end   = [[10,30,2020],[12,3,2019],[1,21,2019],[11,16,2018],[3,22,2018],[3,15,2018],[3,8,2018],[3,3,2018],[1,5,2018],
                [3,15,2017],[2,10,2017],[11,22,2016],[1,24,2016],[2,16,2015],[2,10,2015],[2,3,2015],[1,28,2015],
                [12,14,2014],[11,28,2014],[2,14,2014],[1,22,2014],[1,3,2014],[12,16,2013],[3,20,2013],[3,9,2013],
                [2,10,2013],[12,31,2012],[12,28,2012],[10,31,2011],[2,27,2011],[2,4,2011],[1,27,2011],[1,13,2011],
                [12,28,2010],[3,1,2010],[2,19,2010],[2,11,2010],[2,8,2010],[1,4,2010],[12,21,2009],[12,11,2009],
                [3,3,2009],[2,24,2009],[12,23,2008],[12,22,2008],[12,17,2007],[12,4,2007],[4,6,2007],[3,18,2007],
                [2,16,2007],[2,14,2006],[3,2,2005],[1,24,2005], [12,8,2003],[2,18,2003],[1,4,2003],[12,26,2002],[1,1,2001],[2,20,2000],[2,1,2000],[1,27,2000],
                [3,16,1999],[1,16,1999],[4,1,1997],[1,12,1997],[4,11,1996],[3,9,1996],[2,4,1996],[1,9,1996],
                [12,22,1995],[2,6,1995],[3,4,1994],[2,25,1994],[1,18,1994],[1,9,1994],[1,5,1994],[3,15,1993],
                [2,24,1993],[2,18,1993],[12,13,1992],[12,29,1990],[2,14,1988],[1,27,1988],[1,9,1988],[12,17,1987],
                [11,12,1987],[2,24,1987],[1,24,1987],[1,12,1987],[1,3,1987],[3,5,1985],[2,3,1985],[3,1,1984],
                [2,13,1983],[4,8,1982],[1,24,1982],[1,15,1982],[2,20,1979],[2,9,1979],[2,8,1978],[1,21,1978],
                [1,19,1978],[1,15,1978],[3,25,1977],[1,11,1977],[1,12,1974],[12,18,1973],[2,20,1972],[11,27,1971],
                [3,6,1971],[1,2,1971],[12,29,1969],[3,4,1969],[2,28,1969],[2,10,1969],[3,23,1967],[2,8,1967],
                [12,26,1966],[2,26,1966],[2,1,1966],[1,24,1966],[2,21,1964],[1,14,1964],[12,24,1963],[3,9,1962],
                [3,7,1962],[2,16,1962],[2,5,1961],[1,21,1961],[12,13,1960],[3,5,1960],[2,15,1960],[3,14,1959],
                [3,23,1958],[2,18,1958],[12,5,1957],[3,20,1956],[3,17,1956],[3,9,1956],[2,18,1952],[12,16,1951],
                [11,30,1950],[2,17,1950],[2,1,1949],[1,25,1948],[12,27,1947],[3,4,1947],[2,24,1947],[2,21,1946],
                [12,20,1945],[1,17,1945],[12,13,1944],[2,13,1944],[1,29,1943],[3,31,1942],[3,4,1942],[3,10,1941],
                [2,15,1940],[11,25,1938],[2,15,1936],[1,21,1936],[1,25,1935],[2,27,1934],[12,27,1933],[12,18,1932],
                [3,12,1931],[12,24,1929],[2,22,1929],[2,21,1927],[2,5,1926],[1,10,1926],[1,31,1925],[1,3,1925],
                [4,4,1924],[2,21,1924],[2,7,1923],[1,30,1922],[2,22,1921],[2,7,1920],[1,29,1918],[1,23,1918],
                [1,16,1918],[12,15,1917],[12,9,1917],[3,6,1917],[3,9,1916],[12,15,1915],[4,5,1915],[3,8,1915],
                [2,3,1915],[2,15,1914],[2,18,1910],[2,13,1910],[1,15,1910],[12,27,1909],[3,5,1909],[1,31,1909],
                [1,15,1909],[2,20,1908],[2,7,1908],[2,2,1908],[2,6,1907],[3,20,1906],[3,17,1906],[1,30,1904],
                [2,18,1903],[12,14,1902],[3,6,1902],[2,19,1902],[2,6,1901],[3,16,1900],[3,3,1900]]
for i in range(len(event_end)):
    # Window of analysis
    D_Start   = datetime(event_start[i][2],event_start[i][0],event_start[i][1]).date()
    D_Start   = YYYYMMDD_string(D_Start) 
    D_End     = datetime(event_end[i][2],event_end[i][0],event_end[i][1]).date()
    D_End     = YYYYMMDD_string(D_End)
    #CSV_PRINT = YYYYMMDD_string(D_Start) + "_" + YYYYMMDD_string(D_End)
    print(D_Start[0:4]+"-"+D_Start[4:6]+"-"+D_Start[6:8]+"_"+D_End[0:4]+"-"+D_End[4:6]+"-"+D_End[6:8])

2020-10-29_2020-10-30
2019-11-29_2019-12-03
2019-01-18_2019-01-21
2018-11-14_2018-11-16
2018-03-20_2018-03-22
2018-03-11_2018-03-15
2018-03-05_2018-03-08
2018-03-01_2018-03-03
2018-01-03_2018-01-05
2017-03-12_2017-03-15
2017-02-09_2017-02-10
2016-11-17_2016-11-22
2016-01-22_2016-01-24
2015-02-14_2015-02-16
2015-02-08_2015-02-10
2015-01-29_2015-02-03
2015-01-25_2015-01-28
2014-12-09_2014-12-14
2014-11-26_2014-11-28
2014-02-11_2014-02-14
2014-01-20_2014-01-22
2013-12-30_2014-01-03
2013-12-13_2013-12-16
2013-03-17_2013-03-20
2013-03-03_2013-03-09
2013-02-08_2013-02-10
2012-12-28_2012-12-31
2012-12-24_2012-12-28
2011-10-25_2011-10-31
2011-02-24_2011-02-27
2011-02-01_2011-02-04
2011-01-26_2011-01-27
2011-01-09_2011-01-13
2010-12-24_2010-12-28
2010-02-21_2010-03-01
2010-02-12_2010-02-19
2010-02-08_2010-02-11
2010-02-04_2010-02-08
2009-12-28_2010-01-04
2009-12-18_2009-12-21
2009-12-07_2009-12-11
2009-02-26_2009-03-03
2009-02-22_2009-02-24
2008-12-21_2008-12-23
2008-12-18_2008-12-22
2007-12-14

In [72]:
import pandas as pd
import numpy as np 
import math
import re

###GIVEN ONE POINT, A DIRECTION, AND DISTANCE, WHAT IS THE COORDINATES OF THE SECOND POINT?
def New_Coordinates(lon_o,lat_o,dir_o,dist_o):
    R    = 6378.1                           #Radius of the Earth
    brng = DEGREES[dir_o==BEARINGS][0]      #Bearing is 90 degrees converted to radians.
    d    = float(dist_o)*1.60934                   #Distance in km
    lat1 = math.radians(float(lat_o))              #Current lat point converted to radians
    lon1 = math.radians(float(lon_o))              #Current long point converted to radians
    lat2 = math.asin( math.sin(lat1)*math.cos(d/R)+math.cos(lat1)*math.sin(d/R)*math.cos(brng))
    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),math.cos(d/R)-math.sin(lat1)*math.sin(lat2))
    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)
    return(lon2,lat2)
#----------------------------------------------------------------------------------------------------------------
#Process
BEARINGS    = np.asarray(['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW'])
DEGREES     = np.arange(0,360,45/2)
STATES      = np.asarray(['MAINE','NEW HAMPSHIRE','VERMONT','MASSACHUSETTS',
                          'RHODE ISLAND','CONNECTICUT','NEW YORK','NEW JERSEY',
                          'PENNSYLVANIA','OHIO','WEST VIRGINIA','MARYLAND',
                          'DELAWARE','VIRGINIA'])

STATIONS    = pd.read_csv('C:/Users/Mike/Desktop/GHCND/PNS_Stations_List.csv')
PRINT       = 'PNS_20181131_'
IEM_DATA_pd = pd.read_csv('C:/Users/Mike/Desktop/GHCND/IEM_PNS/PNS_20181131.csv', header=None)
#Clean IEM pd
#(TOWN STATE COUNTY),DISTANCE,BEARING,AMOUNT
IEM_DATA_np  = np.empty([len(IEM_DATA_pd),4], dtype=object)
for i in range(len(IEM_DATA_pd)):
    IEM_DATA_pd_i = IEM_DATA_pd[0][i]
    if IEM_DATA_pd_i in STATES:  STATE  = IEM_DATA_pd_i.strip()                     #OBTAIN THE STATE INFORMATION
    elif '...' in IEM_DATA_pd_i: COUNTY = IEM_DATA_pd_i.replace('...','').strip()   #OBTAIN THE COUNTY INFORMATION
    else:                                                                           #CREATE THE JOIN ARRAY
        IEM_DATA_pd_i = IEM_DATA_pd[0][i].strip().split(" ")                        #IF NO BEARING OR DISTANCE IS GIVEN
        if len(re.sub('\D', '', IEM_DATA_pd_i[0]))==0:
            DISTANCE  = 'NONE'
            DIRECTION = 'NONE'
            TOWN_list = ''
            for j in range(len(IEM_DATA_pd_i)):
                if (''==IEM_DATA_pd_i[j]) or (len(re.sub('\D', '', IEM_DATA_pd_i[j]))==0):
                    TOWN_list += IEM_DATA_pd_i[j]+' '
                else: 
                    AMNT = IEM_DATA_pd_i[j]
                    break 
            IEM_DATA_np[i,0] = TOWN_list.strip()+' '+STATE+' '+COUNTY
            IEM_DATA_np[i,1] = DIRECTION
            IEM_DATA_np[i,2] = DISTANCE
            IEM_DATA_np[i,3] = AMNT
        else:                                                                      #IF BEARING AND DISTANCE IS GIVEN
            DISTANCE      = IEM_DATA_pd_i[0]
            DIRECTION     = IEM_DATA_pd_i[1]
            TOWN_list     = ''
            IEM_DATA_pd_i = IEM_DATA_pd_i[2:]
            for j in range(len(IEM_DATA_pd_i)):
                if (''==IEM_DATA_pd_i[j]) or (len(re.sub('\D', '', IEM_DATA_pd_i[j]))==0):
                    TOWN_list += IEM_DATA_pd_i[j]+' '
                else: 
                    AMNT = IEM_DATA_pd_i[j]
                    break 
            IEM_DATA_np[i,0] = TOWN_list.strip()+' '+STATE+' '+COUNTY
            IEM_DATA_np[i,1] = DIRECTION
            IEM_DATA_np[i,2] = DISTANCE
            IEM_DATA_np[i,3] = AMNT
#CLEAN THE MATRIX AND TURN INTO PANDAS DF
IEM_DATA_np      = IEM_DATA_np[np.sum(IEM_DATA_np==None,axis=1)!=4]
IEM_DATA_pd      = pd.DataFrame(IEM_DATA_np)
IEM_DATA_pd.rename(columns={0:'Location',1:'Bearing',2:'Distance',3:'Snowfall_inch'}, inplace=True)
#JOIN COORDINATES
IEM_Joined_pd    = IEM_DATA_pd.merge(STATIONS, on='Location', how='inner').reset_index(drop=True)
#CHECK JOIN
Joined_Locations = IEM_Joined_pd['Location']
Availa_Locations = IEM_DATA_pd['Location']
Logica_Locations = []
print("There are "+str(len(Availa_Locations)-len(IEM_Joined_pd))+" Missing Stations!!!")
for i in range(len(Availa_Locations)):
    if Availa_Locations[i] in np.asarray(Joined_Locations): pass
    else: print(Availa_Locations[i]) 
print("Copy and paste the above stations to: https://geocode.localfocus.nl/")
print("Add to stations list")
#CORRECT FOR BEARING AND DIRECTION
for i in range(len(IEM_Joined_pd)):
    if IEM_Joined_pd['Bearing'][i] != 'NONE':
        lon_f,lat_f = New_Coordinates(IEM_Joined_pd['Lon'][i],IEM_Joined_pd['Lat'][i],
                                      IEM_Joined_pd['Bearing'][i],IEM_Joined_pd['Distance'][i])
        IEM_Joined_pd['Lat'][i] = str(lat_f)
        IEM_Joined_pd['Lon'][i] = str(lon_f)
IEM_Joined_pd = IEM_Joined_pd[['Lon','Lat','Snowfall_inch']]
IEM_Joined_pd = IEM_Joined_pd.drop_duplicates()
IEM_Joined_pd.to_csv(PRINT+"processed.csv", sep=',',index=False)

There are 2 Missing Stations!!!
Bangor Fss MAINE Penobscot County
Franklin MAINE Washington County
Copy and paste the above stations to: https://geocode.localfocus.nl/
Add to stations list


C:\Users\Mike\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Mike\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
LIST_DIR = pd.read_csv('C:/Users/Mike/NESIS_Kriging/Final_Interpolations/Listing.csv')
TEXT_DIR = 'C:/Users/Mike/NESIS_Kriging/Final_Interpolations/grid_txt/'
for i in range(len(LIST_DIR)):
    DIR_LISTING  = TEXT_DIR+LIST_DIR['Folder'][i]
    TEXT_LISTING = find_files( DIR_LISTING, suffix=".txt" )
    #print(TEXT_LISTING)

['Krig_hole-effect_20191129_20191203.txt']
['Krig_hole-effect_20190118_20190121.txt']
['Krig_spherical_0.25_20181114_20181116.txt']
['Krig_hole-effect_20180320_20180322.txt']
['Krig_hole-effect_20180311_20180315.txt']
['Krig_spherical_0.2_20180305_20180308.txt']
['Krig_hole-effect_20180301_20180303.txt', 'NOTES.txt']
['Krig_spherical_0.2_20180103_20180105.txt']
['Krig_hole-effect_20170312_20170315.txt']
['Krig_spherical_0.6_20170209_20170210.txt']
['Krig_hole-effect_20161117_20161122.txt', 'NOTES.txt']
['Krig_hole-effect_20160122_20160124.txt']
['Krig_hole-effect_20150214_20150216.txt']
['Krig_hole-effect_20150208_20150210.txt']
['Krig_spherical_0.25_20150129_20150203.txt']
['Krig_hole-effect_20150125_20150128.txt']
['Krig_hole-effect_20141209_20141214.txt']
['Krig_hole-effect_20141126_20141128.txt']
['Krig_hole-effect_20140211_20140214.txt']
['Krig_spherical_0.2_20140120_20140122.txt']
['Krig_20131230_20140103.txt']
['Krig_hole-effect_20131213_20131216.txt']
['Krig_spherical_0.25_2013